In [133]:
import pandas as pd
import numpy as np
import itertools
import json

In [134]:
def read_tracking(traintest, lab_id, video_id, pix_per_cm, drop_body_parts=[]):
    """
    Output: DataFrame pvid
    Shape: (num_frames, (num_mice * num_bodyparts *2))
    Mỗi hàm = 1 frame, mỗi cột = toạ độ x/y của 1 bodypart của 1 chuột
    """
    video_path = f"D:/UET/ML/mouse_behavior/data/{traintest}_tracking/{lab_id}/{video_id}.parquet"
    df = pd.read_parquet(video_path)

    if len(np.unique(df.bodypart)) > 5:
        df = df[~df.bodypart.isin(drop_body_parts)]

    # Pivot: frame_idx = idx, columns = mouse_id và (x, y) của bodyparts
    pvid = df.pivot(index="video_frame", columns=["mouse_id", "bodypart"], values=["x", "y"])
    pvid = pvid.reorder_levels([1, 2, 0], axis=1).T.sort_index().T
    pvid.columns = pvid.columns.set_names(["mouse_id", "bodypart", "coord"])

    # Chuấn hoá pixel -> cm
    pvid /= pix_per_cm

    mouse_ids = pvid.columns.get_level_values("mouse_id").unique()
    pvid.attrs["mouse_ids"] = mouse_ids
    pvid.attrs["num_mice"] = len(mouse_ids)
    return pvid

In [135]:
def create_single_mouse_dfs(pvid, annot=None):
    """
    Trả về: list các DataFrame, mỗi DF ứng với 1 chuột
    """
    single_dfs = []
    mouse_ids = pvid.attrs["mouse_ids"]

    # Annotation self-oriented
    if annot is not None:
        annot_self = annot[annot.agent_id == annot.target_id]
    else:
        annot_self = None

    for mouse_id in mouse_ids:
        # Features tracking của chuột
        mouse_features = pvid[mouse_id].copy()

        # Meta info
        meta = pd.DataFrame({
            "video_frame": pvid.index,
            "agent_id": mouse_id,
            "target_id": "self"
        }, index=pvid.index)

        # Build labels
        if annot_self is not None:
            subset = annot_self[annot_self.agent_id == mouse_id]

            actions = subset["action"].unique()
            labels = pd.DataFrame(0, index=pvid.index, columns=actions)

            for _, row in subset.iterrows():
                labels.loc[row.start_frame:row.stop_frame, row.action] = 1
        else:
            labels = None

        # Gom vào 1 dataframe
        if labels is not None:
            df_mouse = pd.concat([mouse_features, meta, labels], axis=1)
        else:
            df_mouse = pd.concat([mouse_features, meta], axis=1)

        # Thêm vào list
        single_dfs.append(df_mouse)

    return single_dfs

def create_mouse_pair_dfs(pvid, annot=None):
    """
    Trả về: list các DataFrame, mỗi DF ứng với 1 cặp agent-target
    """
    pair_dfs = []

    # Annotation social-oriented
    if annot is not None:
        annot_pair = annot[annot.agent_id != annot.target_id]
    else:
        annot_pair = None

    # Get unique mouse IDs
    mouse_ids = np.unique(pvid.columns.get_level_values("mouse_id"))

    for agent, target in itertools.permutations(mouse_ids, 2):
        # Features tracking của agent-target
        pair_features = pvid.loc[:, pvid.columns.get_level_values("mouse_id").isin([agent, target])]

        mapping = {agent: "A", target: "B"}
        pair_features.columns = pd.MultiIndex.from_tuples(
            [(mapping[m], bodypart, coord) for m, bodypart, coord in pair_features.columns]
        )

        # Meta info
        meta = pd.DataFrame({
            "video_frame": pvid.index,
            "agent_id": agent,
            "target_id": target
        }, index=pvid.index)

        # Build labels
        if annot_pair is not None:
            subset = annot_pair[(annot_pair.agent_id==agent) & (annot_pair.target_id==target)]
            actions = subset["action"].unique()
            labels = pd.DataFrame(0, index=pvid.index, columns=actions)
            for _, row in subset.iterrows():
                labels.loc[row.start_frame:row.stop_frame, row.action] = 1
            pair_features = pd.concat([pair_features, meta, labels], axis=1)
        else:
            pair_features = pd.concat([pair_features, meta], axis=1)

        pair_dfs.append(pair_features)

    return pair_dfs

In [136]:
def calculate_centers(df):
    """
    Ensure that 'body_center' exists for all mice or bodyparts combinations.
    Handles both 2-level (bodypart, coord) and 3-level (mouse_id, bodypart, coord) columns.
    
    Fallback logic:
    1. If nose and tail_base exist → midpoint(nose, tail_base)
    2. Else if head and tail_base exist → midpoint(head, tail_base)
    3. Else if only tail_base exists → use tail_base
    4. Else → cannot compute body_center
    """
    cols = df.columns

    # --- 2-level columns (bodypart, coord) ---
    if cols.nlevels == 2:
        if ("body_center", "x") not in df.columns or ("body_center", "y") not in df.columns:
            if ("nose", "x") in df.columns and ("tail_base", "x") in df.columns:
                df[("body_center", "x")] = (df[("nose", "x")] + df[("tail_base", "x")]) / 2
                df[("body_center", "y")] = (df[("nose", "y")] + df[("tail_base", "y")]) / 2
            elif ("head", "x") in df.columns and ("tail_base", "x") in df.columns:
                df[("body_center", "x")] = (df[("head", "x")] + df[("tail_base", "x")]) / 2
                df[("body_center", "y")] = (df[("head", "y")] + df[("tail_base", "y")]) / 2
            elif ("tail_base", "x") in df.columns:
                df[("body_center", "x")] = df[("tail_base", "x")]
                df[("body_center", "y")] = df[("tail_base", "y")]
            else:
                # no valid bodyparts → fill NaN
                df[("body_center", "x")] = np.nan
                df[("body_center", "y")] = np.nan

    # --- 3-level columns (mouse_id, bodypart, coord) ---
    elif cols.nlevels == 3:
        mice = sorted(list(set(c[0] for c in cols)))

        for m in mice:
            has_body_center = ((m, "body_center", "x") in cols) and ((m, "body_center", "y") in cols)
            if not has_body_center:
                if ((m, "nose", "x") in cols) and ((m, "tail_base", "x") in cols):
                    df[(m, "body_center", "x")] = (df[(m, "nose", "x")] + df[(m, "tail_base", "x")]) / 2
                    df[(m, "body_center", "y")] = (df[(m, "nose", "y")] + df[(m, "tail_base", "y")]) / 2
                elif ((m, "head", "x") in cols) and ((m, "tail_base", "x") in cols):
                    df[(m, "body_center", "x")] = (df[(m, "head", "x")] + df[(m, "tail_base", "x")]) / 2
                    df[(m, "body_center", "y")] = (df[(m, "head", "y")] + df[(m, "tail_base", "y")]) / 2
                elif ((m, "tail_base", "x") in cols):
                    df[(m, "body_center", "x")] = df[(m, "tail_base", "x")]
                    df[(m, "body_center", "y")] = df[(m, "tail_base", "y")]
                else:
                    df[(m, "body_center", "x")] = np.nan
                    df[(m, "body_center", "y")] = np.nan
    return df

def calculate_speed_lag(df, part, fps, lag=10, mouse=None):
    cols = df.columns
    if mouse is not None:
        x = df[(mouse, part, "x")]
        y = df[(mouse, part, "y")]
    else:
        x = df[(part, "x")]
        y = df[(part, "y")]

    if x.isna().all() or y.isna().all():
        # all missing → return zeros
        return pd.Series(0, index=df.index)

    dx = x.diff(lag)
    dy = y.diff(lag)
    speed = np.sqrt(dx**2 + dy**2) * fps
    return speed.fillna(0)

# Tính các thống kê của 1 đại lượng theo nhiều cửa sổ thời gian
def calculate_window_stats(df, metric, name, fps, scales=[20, 40, 60, 80]):
    """
    Thêm rolling statistics cho bất kỳ series nào.
    
    metric : pd.Series (ví dụ speed, distance, curvature...)
    fps    : frames_per_second
    scales : list window sizes quy đổi theo 30fps → mặc định [20,40,60,80]
    """
    res = pd.DataFrame(index=df.index)
    for scale in scales:
        ws = max(1, int(round(scale * float(fps) / 30)))
        roll = metric.rolling(ws, min_periods=max(1, ws//4))

        res[f"{name}_mean_{scale}"] = roll.mean()
        res[f"{name}_std_{scale}"]  = roll.std()
        res[f"{name}_min_{scale}"]  = roll.min()
        res[f"{name}_max_{scale}"]  = roll.max()

    return res

In [137]:
def build_single_features(single_mouse_df, meta_fps):
    df_features = pd.DataFrame(index=single_mouse_df.index)
    
    # Get actual bodypart columns
    bodyparts = [col[0] for col in single_mouse_df.columns if isinstance(col, tuple)]
    
    single_mouse_df = calculate_centers(single_mouse_df)
    
    # === Shape and Position Features ===
    # Euclidean distances between bodyparts
    distances = pd.DataFrame({
        f"{p1}+{p2}": np.sqrt(
            (single_mouse_df[(p1, "x")] - single_mouse_df[(p2, "x")])**2 + 
            (single_mouse_df[(p1, "y")] - single_mouse_df[(p2, "y")])**2
        )
        for p1, p2 in itertools.combinations(bodyparts, 2)
    }, index=single_mouse_df.index)
    
    df_features = pd.concat([df_features, distances], axis=1)
    
    # Elongation (only if required bodyparts exist)
    if "nose" in bodyparts and "tail_base" in bodyparts and "ear_left" in bodyparts and "ear_right" in bodyparts:
        df_features["elong"] = distances["nose+tail_base"] / distances["ear_left+ear_right"]
    
    # Body angle (only if nose, tail_base, body_center exist)
    if all(bp in bodyparts for bp in ["nose", "tail_base", "body_center"]):
        v1_x = single_mouse_df[("nose","x")] - single_mouse_df[("body_center","x")]
        v1_y = single_mouse_df[("nose","y")] - single_mouse_df[("body_center","y")]
        v2_x = single_mouse_df[("tail_base","x")] - single_mouse_df[("body_center","x")]
        v2_y = single_mouse_df[("tail_base","y")] - single_mouse_df[("body_center","y")]
        df_features["body_angle"] = (v1_x*v2_x + v1_y*v2_y) / (np.sqrt(v1_x**2+v1_y**2) * np.sqrt(v2_x**2+v2_y**2) + 1e-6)
    
    # === Movement Features ===
    if "body_center" in bodyparts:
        df_features["speed"] = np.sqrt(
            single_mouse_df[("body_center", "x")].diff()**2 +
            single_mouse_df[("body_center", "y")].diff()**2
        )
        df_features["accelerate"] = df_features["speed"].diff()
    
    # Speed lag features for available bodyparts
    for p in ["body_center", "ear_left", "ear_right"]:
        if p in bodyparts:
            df_features[f"speed_{p}_lag_10"] = calculate_speed_lag(single_mouse_df, p, fps=meta_fps)
    
    # Rolling stats for available bodyparts
    for part in bodyparts:
        speed = np.sqrt(single_mouse_df[(part, "x")].diff()**2 + single_mouse_df[(part, "y")].diff()**2) * float(meta_fps)
        res = calculate_window_stats(single_mouse_df, speed, f"speed_{part}", fps=meta_fps)
        df_features = pd.concat([df_features, res], axis=1)
    
    # Curvature for available bodyparts
    for p in ["body_center", "ear_left", "ear_right"]:
        if p in bodyparts:
            lag = max(1, int(round(10 * float(meta_fps) / 30)))
            shifted_x = single_mouse_df[(p,"x")].shift(lag)
            shifted_y = single_mouse_df[(p,"y")].shift(lag)
            df_features[f"curvature_{p}_lag_{lag}"] = np.sqrt((shifted_x - single_mouse_df[(p,"x")])**2 + 
                                                              (shifted_y - single_mouse_df[(p,"y")])**2)
    
    return df_features.fillna(0)


def build_pair_features(mouse_pair_df, meta_fps):
    df_features = pd.DataFrame(index=mouse_pair_df.index)
    
    # Get bodyparts for both mice
    bodyparts_A = [c[1] for c in mouse_pair_df.columns if isinstance(c, tuple) and c[0] == "A"]
    bodyparts_B = [c[1] for c in mouse_pair_df.columns if isinstance(c, tuple) and c[0] == "B"]
    
    mouse_pair_df = calculate_centers(mouse_pair_df)
    
    # Pairwise distances
    distances = pd.DataFrame({
        f"A_{p1}+B_{p2}": np.sqrt(
            (mouse_pair_df[("A", p1, "x")] - mouse_pair_df[("B", p2, "x")])**2 +
            (mouse_pair_df[("A", p1, "y")] - mouse_pair_df[("B", p2, "y")])**2
        )
        for p1, p2 in itertools.product(bodyparts_A, bodyparts_B)
    }, index=mouse_pair_df.index)
    
    df_features = pd.concat([df_features, distances], axis=1)
    
    # Relative orientation (only if nose and tail_base exist for both)
    if all(bp in bodyparts_A for bp in ["nose","tail_base"]) and all(bp in bodyparts_B for bp in ["nose","tail_base"]):
        vec_A_x = mouse_pair_df[("A", "nose", "x")] - mouse_pair_df[("A", "tail_base", "x")]
        vec_A_y = mouse_pair_df[("A", "nose", "y")] - mouse_pair_df[("A", "tail_base", "y")]
        vec_B_x = mouse_pair_df[("B", "nose", "x")] - mouse_pair_df[("B", "tail_base", "x")]
        vec_B_y = mouse_pair_df[("B", "nose", "y")] - mouse_pair_df[("B", "tail_base", "y")]
        df_features["relative_orientation"] = (vec_A_x*vec_B_x + vec_A_y*vec_B_y) / (
            np.sqrt(vec_A_x**2 + vec_A_y**2) * np.sqrt(vec_B_x**2 + vec_B_y**2) + 1e-6
        )
    else: 
        vec_A_x = mouse_pair_df[("A", "head", "x")] - mouse_pair_df[("A", "tail_base", "x")]
        vec_A_y = mouse_pair_df[("A", "head", "y")] - mouse_pair_df[("A", "tail_base", "y")]
        vec_B_x = mouse_pair_df[("B", "head", "x")] - mouse_pair_df[("B", "tail_base", "x")]
        vec_B_y = mouse_pair_df[("B", "head", "y")] - mouse_pair_df[("B", "tail_base", "y")]
        df_features["relative_orientation"] = (vec_A_x*vec_B_x + vec_A_y*vec_B_y) / (
            np.sqrt(vec_A_x**2 + vec_A_y**2) * np.sqrt(vec_B_x**2 + vec_B_y**2) + 1e-6
        )
    
    # Center distance and approach
    if "body_center" in bodyparts_A and "body_center" in bodyparts_B:
        dist_center = np.sqrt(
            (mouse_pair_df[("A", "body_center", "x")] - mouse_pair_df[("B", "body_center", "x")])**2 +
            (mouse_pair_df[("A", "body_center", "y")] - mouse_pair_df[("B", "body_center", "y")])**2
        )
        approach = dist_center.diff().fillna(0)
        df_features["approach_A"] = approach
        df_features["approach_B"] = approach
        
        # Relative distance stats
        res = calculate_window_stats(mouse_pair_df, dist_center**2, "center_distance", fps=meta_fps)
        df_features = pd.concat([df_features, res], axis=1)
        
        # Relative speed
        speed_A = calculate_speed_lag(mouse_pair_df, "body_center", meta_fps, mouse="A")
        speed_B = calculate_speed_lag(mouse_pair_df, "body_center", meta_fps, mouse="B")
        df_features["speed_A_lag_10"] = speed_A
        df_features["speed_B_lag_10"] = speed_B
        df_features["relative_speed_A_B_lag_10"] = (speed_A - speed_B).abs()
    
    return df_features.fillna(0)

In [138]:
drop_body_parts =  [
    'headpiece_bottombackleft', 'headpiece_bottombackright', 'headpiece_bottomfrontleft', 'headpiece_bottomfrontright', 
    'headpiece_topbackleft', 'headpiece_topbackright', 'headpiece_topfrontleft', 'headpiece_topfrontright', 
    'spine_1', 'spine_2', 'tail_middle_1', 'tail_middle_2', 'tail_midpoint'
]

def generate_mouse_data(dataset, traintest, drop_body_parts=[]):
    """
    Tạo DataFrames cho mỗi chuột và mỗi cặp chuột trong 1 video
    Trả về danh sách DataFrames
    """
    all_single = []
    all_pair = []

    for _, row in dataset.iterrows():
        # Meta info
        lab_id = row.lab_id
        video_id = row.video_id
        pix_per_cm = row.pix_per_cm_approx
        meta_fps = row.frames_per_second
        
        # Đọc tracking
        pvid = read_tracking(traintest, lab_id, video_id, pix_per_cm, drop_body_parts=drop_body_parts)
        
        # Đọc annotation
        if traintest == "train":
            annot_path = f"D:/UET/ML/mouse_behavior/data/{traintest}_annotation/{row.lab_id}/{video_id}.parquet"
            annot = pd.read_parquet(annot_path)
        else:
            annot = None
            
        # Tạo DataFrame cho từng chuột
        sdfs = create_single_mouse_dfs(pvid, annot)

        for df_single in sdfs:
            df_features = build_single_features(df_single, meta_fps)
            all_single.append(df_features)

        # Nếu có nhiều hơn 1 chuột, tạo DataFrame cho từng cặp chuột
        if pvid.attrs["num_mice"] > 1:
            pdfs = create_mouse_pair_dfs(pvid, annot)
            for df_pair in pdfs:
                df_features = build_pair_features(df_pair, meta_fps)
                all_pair.append(df_features)

    return all_single, all_pair

In [139]:
train_csv = pd.read_csv('D:/UET/ML/mouse_behavior/data/train.csv') 
test_csv = pd.read_csv('D:/UET/ML/mouse_behavior/data/test.csv')
train_annotation_path = 'D:/UET/ML/mouse_behavior/data/train_annotation'
train_tracking_path = 'D:/UET/ML/mouse_behavior/data/train_tracking'

first_row_df = train_csv[train_csv.video_id == 44566106]      # keep as DataFrame with 1 row
all_single, all_pair = generate_mouse_data(first_row_df, "train", drop_body_parts)

In [140]:
print(all_pair)

[             A_body_center+B_body_center  A_body_center+B_ear_left  \
video_frame                                                          
0                              51.840286                  0.000000   
1                              51.806015                  0.000000   
2                              51.477428                  0.000000   
3                              51.935961                  0.000000   
4                              52.012644                 48.497962   
...                                  ...                       ...   
18446                          34.526432                  0.000000   
18447                          34.835901                  0.000000   
18448                          34.692373                  0.000000   
18449                          34.531979                  0.000000   
18450                          34.432470                  0.000000   

             A_body_center+B_ear_right  A_body_center+B_lateral_left  \
video_frame     